In [ ]:
import requests
import os
import sys

import findspark
findspark.init()
findspark.find()

import xml.etree.ElementTree as ET

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [ ]:
def fetch_stations():
    url = "https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/station/HH"
    headers = {
        "DB-Client-Id": "7b61ee043a945260d2fefbcf867ee8c0",
        "DB-Api-Key": "8f4ad462350ee13f932a3aa4a42663b8",
        "accept": "application/xml"
    }
    response = requests.get(url, headers=headers)
    return response.text if response.status_code == 200 else None

In [ ]:
def parse_stations(xml_data):
    root = ET.fromstring(xml_data)
    # for station in root.findall("station"):
    #     yield (
    #         station.get("meta"), station.get("name"), station.get("eva"),
    #         station.get("ds100"), station.get("db"), station.get("creationts")
    #     )

    stations = []
    for station in root.findall("station"):
        stations.append((
            station.get("meta"),
            station.get("name"),
            station.get("eva"),
            station.get("ds100"),
            station.get("db"),
            station.get("creationts")
        ))
    return stations

In [ ]:
spark= SparkSession \
       .builder \
       .appName("Station Patern") \
       .getOrCreate()

In [ ]:
# schema = T.StructType([
#         T.StructField("meta", T.StringType(), True),
#         T.StructField("name", T.StringType(), True),
#         T.StructField("eva", T.StringType(), True),
#         T.StructField("ds100", T.StringType(), True),
#         T.StructField("db", T.StringType(), True),
#         T.StructField("creationts", T.StringType(), True)
#     ])

ЛИБО:

In [ ]:
StationDatabase = [('meta',T.StringType()), ('name',T.StringType()), ('eva',T.StringType()),
       ('ds100', T.StringType()), ('db', T.StringType()), ('creationts',T.StringType())]

schema = T.StructType([T.StructField(el[0], el[1], True) for el in StationDatabase])


In [ ]:
stations_xml = fetch_stations()

station_rows = parse_stations(stations_xml)

df = spark.createDataFrame(station_rows, schema=schema)

df.write.mode("overwrite").parquet('/stations')
spark.stop()